In [72]:
import requests
import pandas as pd
import os

In [73]:
# Solo se realiza una vez
os.environ['OPENAI_API_KEY'] = '...'
# os.environ['HF_API_KEY'] = '...'

In [ ]:
OPENAI_KEY = os.getenv('OPENAI_API_KEY')
HF_KEY = os.getenv('HF_API_KEY')
headers = {"Authorization": f"Bearer {HF_KEY}"}
print(OPENAI_KEY, HF_KEY)

In [75]:
los_rios = pd.read_csv('noticias-losrios-2023.csv')
test = pd.read_csv('test/dataset-test-desafio2.csv')

<h3 style="color: orange" >Traducción a inglés</h3>

In [76]:
API_URL_TRANSLATE = "https://api-inference.huggingface.co/models/Helsinki-NLP/opus-mt-es-en"

In [77]:
def query_translate(payload):
	response = requests.post(API_URL_TRANSLATE, headers=headers, json=payload)
	return response.json()

In [78]:
import math

def translate_to_en(new: str) -> str:
    size_of_new = len(new)
    iter = math.ceil(size_of_new / 512)
    translated = ''
    for i in range(0, iter):
        start_index = i * 512
        end_index = (i + 1) * 512
        fragment = new[start_index:end_index]

        output = query_translate({
            "inputs": f'{fragment}',
        })
        
        translated += str(output[0]['translation_text'])
    return translated

In [79]:
new = test['text'][21] # probe con 21, 22
result = translate_to_en(new)

In [80]:
print(result)

What started as the dream of two educators who went out to run through the city’s forests, today is a reality. Carola Püschel and Claudia Contreras wondered, how to protect these places so that more people could enjoy them, and their response was children. “As long as they know them and live them, there is no doubt that they will take care of them,” Claudia adds. It was so two years ago, in the middle of a pandemic, and with the urgent need to get the children out of their homes, the opening of theClub del Bosque Valdiviano, kindergarten and workshops. “We were surprised by the high demand for tuition, we realized that there was a community that needed a project like ours,” adds Carola. The project is based on outdoor learning, a philosophy proven years ago in countries such as England, Denmark and Sweden. Precisely based on these experiences, with training abroad and at the national level, the Club del Bosque has been building its own method. “We took what we didIt is best suited to o

<h3 style="color: orange" >Evento principal </h3>

In [81]:
API_URL_EVENT = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"

In [100]:
def query_event(payload):
	response = requests.post(API_URL_EVENT, headers=headers, json=payload)
	return response.json()
	
output = query_event({
	"inputs": result,
})
summary = output[0]["summary_text"]

In [101]:
print(summary)

Two years ago, in the middle of a pandemic, the opening of theClub del Bosque Valdiviano, kindergarten and workshops. The project is based on outdoor learning, a philosophy proven years ago in countries such as England, Denmark and Sweden. In March 2024 the Forest College opens its doors.


<h3 style="color: orange">Ubicación </h3>

In [102]:
from llama_index.llms import HuggingFaceInferenceAPI, HuggingFaceLLM

remotely_run = HuggingFaceInferenceAPI(
    model_name="bigscience/bloom", token=HF_KEY,
)

In [108]:
question_summary=f"What is the main entity mentioned in {summary}?."
completion_response = remotely_run.complete(question_summary)
print(completion_response)

In [85]:
from llama_index import ServiceContext , VectorStoreIndex, SimpleDirectoryReader

service_context = ServiceContext.from_defaults(chunk_size=512, llm=remotely_run)

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [86]:
query_engine = index.as_query_engine()

In [95]:
question=f'For the following summary, extract the most important location mentioned. You must respond in CSV format. Please {summary}'

In [96]:
response = query_engine.query(question)
print(response)

-----------
The answer is the location "Bosque". The location is mentioned in the following sentences:
Bosque Carampangue 190, 5110260 Valdivia, Los Ríos, Chile Tienda del Bosque;
Bosque Las Iris 410, Valdivia, Los Ríos, Chile Servicios Forestales Agricolas Y Transportes El Bosque Limitada;
Bosque Fresia 290, 5110230 Valdivia, Los Ríos, Chile Hostel Bosque Nativo;
Bosque Quillahue 2226, 5090788 Valdivia, Los Ríos, Chile Cabañas EL Bosque;
Bosque Acceso norte - Puente Sta.

file_path: data/result_1.txt

Ramón Picarte 1654, 5090895 Valdivia, Los Ríos, Chile Río de colores;
Jardín Don Bosco 2701, 5100655 Valdivia, Los Ríos, Chile Jardín Infantil y Sala Cuna Carrusel Junji;
Jardín José María Muñoz Hermosilla, 5090238 Valdivia, Los Ríos, Chile Sala cuna y jardín infantil los avellanos;
Jardín Río Loa, 5100936 Valdivia, Los Ríos, Chile Jardín infantil independencia;
Jardín Av. Ramón Picarte 2399
